In [16]:
import numpy as np
import random
import time
import pandas as pd
import operator
from collections import defaultdict
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt

In [17]:
class Node(object):
    def __init__(self, data = None, next_node = None, prev_node = None):
        self.data = data
        self.next_node =  next_node
        self.prev_node = prev_node

class DoublyLinkedList(object):
    def __init__(self, head = None):
        self.head = head
    
    def traverse(self):
        current_node = self.head
        while current_node != None:
            print(current_node.data)
            current_node = current_node.next_node
    
    def get_size(self):
        count = 0
        current_node = self.head
        while current_node != None:
            count += 1
            current_node = current_node.next_node
        return count
            
    def append(self, data):
        new_node = Node(data)
        current_node = self.head
        new_node.next_node = current_node
        new_node.prev_node = None
        if current_node != None:
            current_node.prev_node = new_node
        self.head = new_node
    
    def insert_end(self, data):
        new_node = Node(data)
        new_node.next = None
        if self.head == None:
            new_node.prev_node = None
            self.head = new_node
        return
    
        first_node = self.head
        while first_node.next_node:
            first_node = first_node.next_node
        first_node.next_node = new_node
        new_node.prev_node = first_node
    
    
    def delete(self, data):
        current_node = self.head
        while current_node != None:
            if current_node.data == data and current_node == self.head:
                if not current_node.next_node:
                    current_node = None
                    self.head = None
                    return
                else:
                    q = current_node.next_node
                    current_node.next_node = None
                    q.prev_node = None
                    current_node = None
                    self.head = q
                    return
            
            elif current_node.data == data:
                if current_node.next_node != None:
                    p = current_node.prev_node
                    q = current_node.next_node
                    p.next_node = q
                    q.prev_node = p
                    current_node.next_node = None
                    current_node.prev_node = None
                    current_node = None
                    return
                else:
                    p = current_node.prev_node
                    p.next_node = None
                    current_node.prev_node = None
                    current = None
                    return
            current_node =  current_node.next_node

class Bucket(object):
    def __init__(self, value):
        self.gain = value
        self.data = DoublyLinkedList()

class Bucket_Arrays(object):
    def __init__(self, maxdegree):
        self.bucket_range = maxdegree
        self.left_buckets = [Bucket(i) for i in range(-self.bucket_range,self.bucket_range+1)]
        self.right_buckets = [Bucket(i) for i in range(-self.bucket_range,self.bucket_range+1)]   

In [20]:
def calculate_num_cuts(Partition_A,Partition_B):
    cuts = 0
    for i in Partition_A:
        connected_array = df.iloc[int(i)-1][3]
        for j in connected_array:
            if int(j) not in Partition_A:
                cuts += 1
    return cuts

def initialize_buckets(Partition_A,Partition_B,max_degree):
    bucket_arrays = Bucket_Arrays(max_degree)
    for i in Partition_A:
        gain = 0
        connected_array = df.iloc[int(i)-1][3]
        for j in connected_array:
            if int(j) in Partition_A:
                gain -= 1
            else:
                gain += 1
        df.loc[int(i)-1,'Gain'] = gain
        bucket_arrays.left_buckets[gain+max_degree].data.append(i)
    
    for i in Partition_B:
        gain = 0
        connected_array = df.iloc[int(i)-1][3]
        for j in connected_array:
            if int(j) in Partition_B:
                gain -= 1
            else:
                gain += 1
        df.loc[int(i)-1,'Gain'] = gain
        bucket_arrays.right_buckets[gain+max_degree].data.append(i)
    return bucket_arrays

In [21]:
# Find vertex with maximum gain in buckets
def calculate_vertex_max_gain(buckets):
    for bucket in buckets:
        max_gain_bucket = bucket.data.head
        if(max_gain_bucket != None):
            max_gain = bucket.gain
            vertex_max_gain = max_gain_bucket.data
    return max_gain, vertex_max_gain

def re_calculate_gain(connected_array,Partition,buckets):
    for i in connected_array:
        for k in buckets:
            k.data.delete(int(i))
    for i in connected_array:
        gain = 0
        adjacent_vertices = df.iloc[int(i)-1][3]
        for j in adjacent_vertices:
            if int(j) in Partition:
                gain -= 1
            else:
                gain += 1
        buckets[gain+max_degree].data.append(i)
    return buckets

# Update buckets after vertex move 
def update_buckets(buckets, max_gain,vertex_max_gain, update_gain_left, Partition_A,Partition_B):
    vertex_max_gain = int(vertex_max_gain)
    if(update_gain_left == 0):
        buckets.left_buckets[max_gain+max_degree].data.delete(vertex_max_gain)
        connected_array = df.iloc[vertex_max_gain-1][3]
        buckets.right_buckets = re_calculate_gain(connected_array,Partition_B, buckets.right_buckets)
        
    else:
        buckets.right_buckets[max_gain+max_degree].data.delete(vertex_max_gain)
        connected_array = df.iloc[vertex_max_gain-1][3]
        buckets.left_buckets = re_calculate_gain(connected_array,Partition_A, buckets.left_buckets)
    return buckets

In [22]:
def Fiduccia_Mattheyses_LS():
    start = time.time()
    
    # Create two partitions A and B
    num_vertices = len(df)
    vertices = np.arange(1,501)
    np.random.shuffle(vertices)
    cut = int(0.5 * num_vertices)
    Partition_A = vertices[:cut]
    Partition_B = vertices[cut:]

    max_degree = np.max(df['Number of connected vertices'])
    num_cuts = calculate_num_cuts(Partition_A,Partition_B)
    min_cuts = num_cuts
    buckets = initialize_buckets(Partition_A,Partition_B,max_degree)
    
    while(np.sum(df['Fixed'])<num_vertices):
        if(len(Partition_A)>=len(Partition_B)):
            # Calculate vertex with maximum gain in left bucket
            max_gain, vertex_max_gain = calculate_vertex_max_gain(buckets.left_buckets)

            # Update buckets and partitions
            Partition_A = np.delete(Partition_A, np.where(Partition_A == vertex_max_gain))
            Partition_B = np.append(Partition_B,vertex_max_gain)

            buckets = update_buckets(buckets, max_gain,vertex_max_gain,0, Partition_A, Partition_B)

            # Update fixed vertices and cuts tracker
            fixed_vertices.append(vertex_max_gain)
            cuts = calculate_num_cuts(Partition_A,Partition_B)
            if(cuts < min_cuts):
                save_state = np.array([cuts,Partition_A,Partition_B])
                min_cuts = cuts

        else:

            # Calculate vertex with maximum gain in left bucket
            max_gain, vertex_max_gain = calculate_vertex_max_gain(buckets.right_buckets)

            # Update buckets and partitions
            Partition_B = np.delete(Partition_B, np.where(Partition_B == vertex_max_gain))
            Partition_A = np.append(Partition_A,vertex_max_gain)

            buckets = update_buckets(buckets, max_gain,vertex_max_gain,1, Partition_A, Partition_B)

            # Update fixed vertices and cuts tracker
            fixed_vertices.append(vertex_max_gain)
            cuts = calculate_num_cuts(Partition_A,Partition_B)
            if(cuts < min_cuts):
                save_state = np.array([cuts,Partition_A,Partition_B])
                min_cuts = cuts
        #print(len(fixed_vertices),end='\r')
        print(cuts)
    # optimal_num_cuts = np.min(cuts_tracker[:,0]) 
    # final_partition1 = cuts_tracker[np.argmin(cuts_tracker[:,0]) ][1]
    # final_partition2 = cuts_tracker[np.argmin(cuts_tracker[:,0]) ][2]
    end = time.time()
    elapsed_time = end-start
    print("Time taken", elapsed_time)
    return save_state

In [33]:
num_vertices = len(df)
vertices = np.arange(1,5)
np.random.shuffle(vertices)
cut = int(0.5 * num_vertices)
Partition_A = vertices[:cut]
Partition_B = vertices[cut:]

In [35]:
max_degree = np.max(df['Number of connected vertices'])
num_cuts = calculate_num_cuts(Partition_A,Partition_B)
min_cuts = num_cuts
buckets = initialize_buckets(Partition_A,Partition_B,max_degree)

In [36]:
#buckets.right_buckets[4].data.traverse()

2


In [104]:
#buckets.right_buckets[2].data.traverse()

In [ ]:
%%

In [106]:
max_gain, vertex_max_gain = calculate_vertex_max_gain(buckets.left_buckets)
Partition_A = np.delete(Partition_A, np.where(Partition_A == vertex_max_gain))
Partition_B = np.append(Partition_B,vertex_max_gain)
buckets = update_buckets(buckets, max_gain,vertex_max_gain,0, Partition_A, Partition_B)

In [116]:
buckets.left_buckets[4].data.traverse()

In [121]:
buckets.right_buckets[4].data.traverse()

In [ ]:
if(len(Partition_A)>=len(Partition_B)):
    # Calculate vertex with maximum gain in left bucket
    max_gain, vertex_max_gain = calculate_vertex_max_gain(buckets.left_buckets)

    # Update buckets and partitions
    Partition_A = np.delete(Partition_A, np.where(Partition_A == vertex_max_gain))
    Partition_B = np.append(Partition_B,vertex_max_gain)

    buckets = update_buckets(buckets, max_gain,vertex_max_gain,0, Partition_A, Partition_B)

    # Update fixed vertices and cuts tracker
    fixed_vertices.append(vertex_max_gain)
    cuts = calculate_num_cuts(Partition_A,Partition_B)
    if(cuts < min_cuts):
        save_state = np.array([cuts,Partition_A,Partition_B])
        min_cuts = cuts

In [ ]:
%%

In [ ]:
while(np.sum(df['Fixed'])<num_vertices):
    if(len(Partition_A)>=len(Partition_B)):
        # Calculate vertex with maximum gain in left bucket
        max_gain, vertex_max_gain = calculate_vertex_max_gain(buckets.left_buckets)

        # Update buckets and partitions
        Partition_A = np.delete(Partition_A, np.where(Partition_A == vertex_max_gain))
        Partition_B = np.append(Partition_B,vertex_max_gain)

        buckets = update_buckets(buckets, max_gain,vertex_max_gain,0, Partition_A, Partition_B)

        # Update fixed vertices and cuts tracker
        fixed_vertices.append(vertex_max_gain)
        cuts = calculate_num_cuts(Partition_A,Partition_B)
        if(cuts < min_cuts):
            save_state = np.array([cuts,Partition_A,Partition_B])
            min_cuts = cuts

    else:

        # Calculate vertex with maximum gain in left bucket
        max_gain, vertex_max_gain = calculate_vertex_max_gain(buckets.right_buckets)

        # Update buckets and partitions
        Partition_B = np.delete(Partition_B, np.where(Partition_B == vertex_max_gain))
        Partition_A = np.append(Partition_A,vertex_max_gain)

        buckets = update_buckets(buckets, max_gain,vertex_max_gain,1, Partition_A, Partition_B)

        # Update fixed vertices and cuts tracker
        fixed_vertices.append(vertex_max_gain)
        cuts = calculate_num_cuts(Partition_A,Partition_B)
        if(cuts < min_cuts):
            save_state = np.array([cuts,Partition_A,Partition_B])
            min_cuts = cuts
    #print(len(fixed_vertices),end='\r')
    print(cuts)
# optimal_num_cuts = np.min(cuts_tracker[:,0]) 
# final_partition1 = cuts_tracker[np.argmin(cuts_tracker[:,0]) ][1]
# final_partition2 = cuts_tracker[np.argmin(cuts_tracker[:,0]) ][2]
end = time.time()
elapsed_time = end-start
print("Time taken", elapsed_time)
return save_state

In [38]:
# Loading the single planar graph of 500 vertices
data = defaultdict(list)
for line in open("Graph501.txt"):
    split_line=line.split()
    ID_vertex = split_line[0]
    num_connected_vertices  = split_line[2]
    ID_connected_vertices = split_line[3:]
    if (ID_vertex) not in data.keys():
        data[ID_vertex].append(int(num_connected_vertices))
        data[ID_vertex].append(0)
        data[ID_vertex].append(0)
        data[ID_vertex].append(ID_connected_vertices)
        data[ID_vertex].append(0)
df = pd.DataFrame(data.values(),columns = ['Number of connected vertices','Gain', 'Fixed','ID connected vertices', 'Partition'])

In [39]:
df.head()

,Number of connected vertices,Gain,Fixed,ID connected vertices,Partition
0,1,0,0,[2],0
1,2,0,0,"[1, 3]",0
2,2,0,0,"[2, 4]",0
3,1,0,0,[3],0


0

In [45]:
fin = Fiduccia_Mattheyses_LS()

time elapsed before while 0.12070393562316895
time for one loop 0.04688692092895508
649
time for one loop 0.03839397430419922
643
time for one loop 0.037897348403930664
638
time for one loop 0.0359041690826416
632
time for one loop 0.034906625747680664
627
time for one loop 0.04089045524597168
622
time for one loop 0.04088997840881348
617
time for one loop 0.04089069366455078
610
time for one loop 0.03889608383178711
605
time for one loop 0.03789854049682617
600
time for one loop 0.034906625747680664
595
time for one loop 0.03688406944274902
590
time for one loop 0.034906625747680664
585
time for one loop 0.0359039306640625
578
time for one loop 0.036900997161865234
571
time for one loop 0.042885541915893555
566
time for one loop 0.03889608383178711
562
time for one loop 0.0359041690826416
555
time for one loop 0.0359036922454834
551
time for one loop 0.0359039306640625
545
time for one loop 0.03490638732910156
539
time for one loop 0.03690195083618164
535
time for one loop 0.037927389

C:\Users\Rohit\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\Rohit\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\Rohit\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


time for one loop 0.04787254333496094
1357
time for one loop 0.05083799362182617
1368
time for one loop 0.0638277530670166
1368
time for one loop 0.07082939147949219
1379
time for one loop 0.05583930015563965
1379
time for one loop 0.051885366439819336
1390
time for one loop 0.0608057975769043
1390
time for one loop 0.06136059761047363
1401
time for one loop 0.056847333908081055
1401
time for one loop 0.05086398124694824
1412
time for one loop 0.05285930633544922
1412
time for one loop 0.06283187866210938
1423
time for one loop 0.05186176300048828
1423
time for one loop 0.05684804916381836
1434


KeyboardInterrupt: 